# Soru 1:
In this assignment, you are going to use your model to predict survival of the Titanic disaster. For this assignment, send a link to a Jupyter notebook containing solutions to the following tasks.
* Download Titanic data from Kaggle. The data in the train.csv file meets your need.
* Split your data into training and test sets.
* Predict the survival based on the test data you split by creating your model.
* Is your model's performance satisfactory? Explain.
* Try to improve your model's performance by adding or subtracting some variables.

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv("titanic.csv")
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [3]:
df["Cabin"].fillna(df["Cabin"].mode()[0], inplace=True)
df["Embarked"].fillna(df["Embarked"].mode()[0], inplace=True)
df["Age"]=df.groupby(["Sex"])["Age"].apply(lambda x: x.fillna(x.mode()[0]))

In [ ]:
# Hedef değişken "Survived" kolonudur ve öncelikle özellik olarak kullanacağım kategorik değişkenleri numerik değişkene dönüştürmeliyiz.

In [4]:
pd.options.display.max_columns
pd.set_option("display.max_columns",None)

In [4]:
df["is_male"]=pd.get_dummies(df.Sex, drop_first=True)

In [5]:
embarked_dummy=pd.get_dummies(df.Embarked, drop_first=False)

df=pd.concat([df,embarked_dummy], axis=1)

In [6]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_male,C,Q,S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,B96 B98,S,1,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,B96 B98,S,0,0,0,1


In [127]:
Y=df["Survived"]
X=df[["is_male", "Pclass", "Age"]]

In [128]:
X_train, X_test, Y_train, Y_test =train_test_split(X,Y, test_size=0.20, random_state=465)

In [130]:
log_reg=linear_model.LogisticRegression()
log_reg.fit(X_train,Y_train)

y_test_pred=log_reg.predict(X_test)

print("R-squared train set", log_reg.score(X_train,Y_train))
print("R-squared test set", log_reg.score(X_test,Y_test))

R-squared train set 0.7991573033707865
R-squared test set 0.7877094972067039


Modelin R-squared değeri yaklaşık % 80 oranında hedef değişkenin varyansını açıklamaktadır yani uygunluğu yüksektir diyebiliriz. Ayrıca, train ve test dataları arasındaki R-squared değerleri birbirine yakın olduğu için genelleme açığı düşüktür diyebiliriz. Uygunluğu arttırabilmek için parametreleri değiştirerek tekrar modeli kuralım:

In [67]:
predictions=np.array([0,3,40]).reshape(1,-1)
print(log_reg.predict(predictions))

[1]


Yukarıdaki tahmini yorumlarsam; is_male, Pclass, Age özelliklerine sırayla(0,3,40) değerlerini girdim ve çıkan sonuç 1'dir yani kişi hayatta kalır. Diğer bir ifadeyle, 40 yaşında 3. sınıfta seyahat eden  bir kadın hayatta kalır diyebiliriz.

In [73]:
log_reg_multinomial=LogisticRegression(multi_class='multinomial', solver="lbfgs")

log_reg_multinomial.fit(X_train, Y_train)

print("Multinomial train R-squared:", log_reg_multinomial.score(X_train,Y_train))
print("Multinomial test R-squared:", log_reg_multinomial.score(X_train,Y_train))

Multinomial train R-squared: 0.7865168539325843
Multinomial test R-squared: 0.7865168539325843


Lojistik regresyonda default olarak kullanılan one vs. rest yönteminde train ve test'in uygunluğu birbirine çok yakın çıkmıştır ancak uygunluğu arttırmayı denemek için multinomial yöntemi ile hesaplama yapmayı denedim. Bu yöntemin sonucunda test in R-squared değeri biraz artıp train in R-squared değeri biraz azalarak eşitliğe gelmiştir.

In [74]:
# Farklı özellikler kullanarak tekrar logistic regresyon modeli kuralım:

In [80]:
Y2=df["Survived"]
X2=df[["is_male","C","Q","S","Parch"]]

In [81]:
X2_train, X2_test, Y2_train, Y2_test =train_test_split(X2, Y2, test_size=0.20, random_state=465)

In [90]:
log_reg_2=linear_model.LogisticRegression()
log_reg_2.fit(X2_train,Y2_train)

print(log_reg_2.score(X2_train,Y2_train))
print(log_reg_2.score(X2_test,Y2_test))

0.7865168539325843
0.7932960893854749


In [104]:
y2_test_pred=log_reg_2.predict(X2_test)

prediction2=np.array([0,0,1,0,4]).reshape(1,-1)
print(log_reg_2.predict(prediction2))

[1]


Bir önceki modelin özellikleriyle logistic regresyon modelinin test kümesinin R-squared değeri 0.782 iken; yeni özelliklerle R-squared değeri 0.793 olmuştur; yani modelin uygunluğu biraz artmıştır diyebiliriz.

In [107]:
Y3=df["Survived"]
X3=df[["Parch","Pclass"]]

In [108]:
X3_train, X3_test, Y3_train, Y3_test =train_test_split(X3,Y3, test_size=0.2, random_state=465)

In [109]:
log_reg3=linear_model.LogisticRegression()
log_reg3.fit(X3_train,Y3_train)

print("Train R-squared", log_reg3.score(X3_train,Y3_train))
print("Test R-squared", log_reg3.score(X3_test,Y3_test))

Train R-squared 0.6797752808988764
Test R-squared 0.7039106145251397


Parch ve Pclass özellikleriyle lojistik regresyon sonucunun doğruluğu düşmüştür.

In [116]:
Y4=df["Survived"]
X4=df[["Pclass","is_male","Age","C","Q","S"]]

In [125]:
X4_train,X4_test,Y4_train,Y4_test= train_test_split(X4,Y4, test_size=0.2, random_state=400)

In [131]:
log_reg4=linear_model.LinearRegression()

log_reg4.fit(X4_train,Y4_train)
y4_test_pred=log_reg4.predict(X4_test)

print(log_reg4.score(X4_train,Y4_train))
print(log_reg4.score(X4_test,Y4_test))

0.35552969052224703
0.4964597636104275


"Pclass","is_male","Age","C","Q","S" özellikleri için modelin doğruluğu düşmüştür.